In [3]:
from decouple import config
import pandas as pd
import requests
import os
import re  # Para limpiar los nombres de archivo
from urllib.parse import urlparse, parse_qs # Para analizar los parámetros de la URL

# --- 1. CONFIGURACIÓN ---
excel_file_path = config('PROYECTO_DATA') +"/Certificado/CERTIFICADOS PROMS DOCENTES.xlsx"  # Ruta a tu archivo Excel
base_download_dir = 'Descargas'   # Carpeta principal donde se crearán las subcarpetas

In [4]:
# --- 2. FUNCIÓN PRINCIPAL (MODIFICADA) ---

def descargar_links_de_excel(excel_path, base_dir):
    """
    Lee un archivo Excel, y para cada hoja, descarga los links
    encontrados en una carpeta con el nombre de la hoja.
    
    Versión 2: Optimizada para URLs dinámicas (como ?nombre=...)
    """
    
    print(f"Cargando archivo: {excel_path}")
    
    try:
        all_sheets = pd.read_excel(excel_path, sheet_name=None, header=None)
    except FileNotFoundError:
        print(f"ERROR: No se encontró el archivo {excel_path}")
        return
    except Exception as e:
        print(f"ERROR: No se pudo leer el archivo Excel. ¿Quizás falta 'openpyxl'? (pip install openpyxl). Error: {e}")
        return

    print(f"Encontradas {len(all_sheets)} hojas: {', '.join(all_sheets.keys())}\n")

    for sheet_name, df in all_sheets.items():
        print(f"--- Procesando Hoja: '{sheet_name}' ---")
        
        safe_sheet_name = str(sheet_name).replace(' ', '_').replace('/', '-')
        download_folder = os.path.join(base_dir, safe_sheet_name)
        os.makedirs(download_folder, exist_ok=True)
        print(f"  Carpeta de destino: {download_folder}")

        links_encontrados = 0
        for cell_value in df.stack():
            
            url = str(cell_value)
            if url.startswith('http://') or url.startswith('https://'):
                links_encontrados += 1
                print(f"\n  [Link {links_encontrados}] Encontrado: {url}")
                
                try:
                    # --- Lógica de descarga modificada ---
                    
                    # 1. Realizar la petición
                    print("  Solicitando al servidor...")
                    response = requests.get(url, stream=True, timeout=15)
                    
                    if response.status_code != 200:
                        print(f"  [!] Error {response.status_code} al descargar: {url}")
                        continue
                        
                    # 2. Verificar si es realmente un PDF
                    content_type = response.headers.get('content-type', '').lower()
                    if 'application/pdf' not in content_type:
                        print(f"  [!] Omitido (El enlace no devolvió un PDF, sino '{content_type}'): {url}")
                        continue

                    # 3. Generar un nombre de archivo
                    filename = None
                    try:
                        # Parsear la URL para obtener los parámetros (ej. 'nombre')
                        query_params = parse_qs(urlparse(url).query)
                        
                        if 'nombre' in query_params:
                            # Obtener el primer valor del parámetro 'nombre'
                            name = query_params['nombre'][0]
                            # Limpiar el nombre para que sea un nombre de archivo seguro
                            # "MEDALID QUISPE TARCO" -> "MEDALID_QUISPE_TARCO"
                            safe_name = re.sub(r'[^\w\s-]', '', name).strip() # Quita caracteres raros
                            safe_name = re.sub(r'[-\s]+', '_', safe_name)      # Reemplaza espacios por guiones bajos
                            filename = f"{safe_name}.pdf"
                        
                        elif 'numero' in query_params:
                            # Fallback: si no hay 'nombre', usar 'numero'
                            num = query_params['numero'][0]
                            filename = f"certificado_{num}.pdf"
                            
                    except Exception as e:
                        print(f"  [!] Advertencia: No se pudo parsear el nombre desde la URL. {e}")

                    # Si todo falla, usar un nombre genérico
                    if not filename:
                        filename = f"certificado_descargado_{links_encontrados}.pdf"
                    
                    print(f"  Nombre de archivo generado: {filename}")

                    # 4. Manejar duplicados y guardar
                    filepath = os.path.join(download_folder, filename)
                    count = 1
                    base, ext = os.path.splitext(filename)
                    while os.path.exists(filepath):
                        new_filename = f"{base}_{count}{ext}"
                        filepath = os.path.join(download_folder, new_filename)
                        count += 1
                    
                    # Guardar el archivo
                    with open(filepath, 'wb') as f:
                        for chunk in response.iter_content(chunk_size=8192):
                            f.write(chunk)
                    print(f"  [+] Éxito. Guardado en: {filepath}")

                except requests.exceptions.RequestException as e:
                    print(f"  [!] Error de red o URL inválida: {url} ({e})")
                except Exception as e:
                    print(f"  [!] Error inesperado: {e}")

        if links_encontrados == 0:
            print("  No se encontraron links en esta hoja.")

    print("\n--- Proceso completado ---")

# --- 3. EJECUTAR EL SCRIPT ---
if __name__ == "__main__":
    descargar_links_de_excel(excel_file_path, base_download_dir)

Cargando archivo: D:\Trama/Certificado/CERTIFICADOS PROMS DOCENTES.xlsx
Encontradas 3 hojas: FACILITADOR, ORGANIZADOR, PARTICIPANTE

--- Procesando Hoja: 'FACILITADOR' ---
  Carpeta de destino: Descargas\FACILITADOR

  [Link 1] Encontrado: http://192.168.0.3:8000/certificado/?nombre=SUSABEL ROXANA ITURBE VALDIVIA&calidad=FACILITADOR&fecha=30-09-2025&folio=4&numero=93
  Solicitando al servidor...
  Nombre de archivo generado: SUSABEL_ROXANA_ITURBE_VALDIVIA.pdf
  [+] Éxito. Guardado en: Descargas\FACILITADOR\SUSABEL_ROXANA_ITURBE_VALDIVIA.pdf

  [Link 2] Encontrado: http://192.168.0.3:8000/certificado/?nombre=NOEMY FANI ALARCON CORNEJO&calidad=FACILITADOR&fecha=30-09-2025&folio=4&numero=94
  Solicitando al servidor...
  Nombre de archivo generado: NOEMY_FANI_ALARCON_CORNEJO.pdf
  [+] Éxito. Guardado en: Descargas\FACILITADOR\NOEMY_FANI_ALARCON_CORNEJO.pdf

  [Link 3] Encontrado: http://192.168.0.3:8000/certificado/?nombre=KECIA ESTEFANI ÑAÑEZ MALDONADO&calidad=FACILITADOR&fecha=30-09-202

In [ ]:
import requests
import urllib.parse

# Lista de URLs
urls = [
    "http://127.0.0.1:8000/certificado/?nombre=SUSABEL ROXANA ITURBE VALDIVIA&calidad=FACILITADOR&fecha=30-09-2025&folio=4&numero=93",
    "http://127.0.0.1:8000/certificado/?nombre=NOEMY FANI ALARCON CORNEJO&calidad=FACILITADOR&fecha=30-09-2025&folio=4&numero=94",
    "http://127.0.0.1:8000/certificado/?nombre=KECIA ESTEFANI ÑAÑEZ MALDONADO&calidad=FACILITADOR&fecha=30-09-2025&folio=4&numero=95",
    # (... agrega las demás URLs ...)
]

# Carpeta destino
carpeta = "certificados/"

import os
os.makedirs(carpeta, exist_ok=True)

for url in urls:
    # Extraer parámetros de la URL
    params = urllib.parse.parse_qs(url.split("?")[1])

    # Obtener número y nombre para el archivo
    numero = params.get("numero", ["sin_numero"])[0]
    nombre = params.get("nombre", ["sin_nombre"])[0].replace(" ", "_")

    # Codificar URL correctamente (por tildes, ñ, espacios)
    url_codificada = (
        url.split("?")[0] + "?" + urllib.parse.urlencode(params, doseq=True)
    )

    print(f"Descargando PDF {numero} ...")

    respuesta = requests.get(url_codificada)

    if respuesta.status_code == 200:
        ruta = f"{carpeta}{numero}_{nombre}.pdf"
        with open(ruta, "wb") as f:
            f.write(respuesta.content)
        print(f"✔ Guardado: {ruta}")
    else:
        print(f"✘ Error al descargar {numero}, código HTTP: {respuesta.status_code}")


Descargando PDF 93 ...
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "d:\IRVIN\Python\Python-GERESA\venv\Lib\site-packages\urllib3\connection.py", line 198, in _new_conn
    sock = connection.create_connection(
        (self._dns_host, self.port),
    ...<2 lines>...
        socket_options=self.socket_options,
    )
  File "d:\IRVIN\Python\Python-GERESA\venv\Lib\site-packages\urllib3\util\connection.py", line 85, in create_connection
    raise err
  File "d:\IRVIN\Python\Python-GERESA\venv\Lib\site-packages\urllib3\util\connection.py", line 73, in create_connection
    sock.connect(sa)
    ~~~~~~~~~~~~^^^^
ConnectionRefusedError: [WinError 10061] No se puede establecer una conexión ya que el equipo de destino denegó expresamente dicha conexión

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "d:\IRVIN\Python\Python-GERESA\venv\Lib\site-packages\urllib3\connectionpool.py", line 787, in urlopen
    response = self._make_request(
        conn,
    ...<

: 